In [1]:
# @title 1. Setup & Installation
# Run this cell first to install necessary libraries
!pip install groq --quiet

import os
from groq import Groq

# Get API key securely from environment variable
api_key = os.getenv("GROQ_API_KEY")

if not api_key:
    raise ValueError("Please set the GROQ_API_KEY environment variable.")

# Initialize the Groq client
client = Groq(api_key=api_key)

MODEL_NAME = "llama-3.3-70b-versatile"

def get_completion(prompt, model=MODEL_NAME):
    """Helper function to call the Groq API"""
    try:
        messages = [{"role": "user", "content": prompt}]
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

print(f"✅ Setup Complete. Using model: {MODEL_NAME}")



[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: C:\Users\Yash\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


ImportError: cannot import name 'Sentinel' from 'typing_extensions' (C:\Users\Yash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\typing_extensions.py)

In [9]:
# @title 2. Zero-Shot vs. Few-Shot Prompting

print("--- ZERO-SHOT PROMPTING ---")
zero_shot_prompt = """
Classify the sentiment of the following text into one of these categories: [Tech Support, Sales, Refund, Complaint].
Text: "I've been waiting for my money back for 3 weeks now!"
Category:
"""
# Note: Zero-shot might be chatty (e.g., "The category is Refund")
response_zero = get_completion(zero_shot_prompt)
print(f"Output: {response_zero}\n")


print("--- FEW-SHOT PROMPTING ---")
few_shot_prompt = """
You are a classification bot. Answer with ONLY the category name. Do not explain.

Text: "My computer screen is blue."
Category: Tech Support

Text: "I want to buy the new subscription."
Category: Sales

Text: "This service is terrible and I hate it."
Category: Complaint

Text: "I've been waiting for my money back for 3 weeks now!"
Category:
"""
# Note: Few-shot forces the model to copy the format exactly
response_few = get_completion(few_shot_prompt)
print(f"Output: {response_few}")

--- ZERO-SHOT PROMPTING ---
Output: Category: Refund

--- FEW-SHOT PROMPTING ---
Output: Complaint


In [10]:
# @title 3. Chain of Thought (CoT) Implementation

logic_problem = "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"

print("--- STANDARD PROMPTING ---")
standard_prompt = f"Q: {logic_problem}\nA:"
response_std = get_completion(standard_prompt)
print(f"Output: {response_std}\n")


print("--- CHAIN OF THOUGHT (CoT) PROMPTING ---")
# The magic phrase "Let's think step by step" triggers the model's reasoning capabilities
cot_prompt = f"Q: {logic_problem}\nA: Let's think step by step."
response_cot = get_completion(cot_prompt)
print(f"Output: {response_cot}")

--- STANDARD PROMPTING ---
Output: To find out how many tennis balls Roger has now, we need to add the tennis balls he already had to the ones he bought.

Roger initially had 5 tennis balls.
He bought 2 cans of tennis balls, with 3 tennis balls in each can. So, he bought 2 * 3 = 6 tennis balls.

Now, we add the tennis balls he already had to the ones he bought: 5 + 6 = 11.

So, Roger has 11 tennis balls now.

--- CHAIN OF THOUGHT (CoT) PROMPTING ---
Output: To find out how many tennis balls Roger has now, let's break down the information step by step:

1. **Initial Number of Tennis Balls**: Roger starts with 5 tennis balls.

2. **Additional Tennis Balls Purchased**: He buys 2 more cans of tennis balls. Each can contains 3 tennis balls. So, the total number of tennis balls he buys is 2 cans * 3 tennis balls per can = 6 tennis balls.

3. **Total Number of Tennis Balls Now**: To find the total number of tennis balls Roger has now, we add the initial number of tennis balls to the number of

In [11]:
# @title 4. The Interview Approach (Persona Pattern)

class InterviewerBot:
    def __init__(self):
        # We instruct the model to ASK questions, not just answer
        self.context = "You are an expert Python Developer. Your goal is to write a script for the user. But first, you must ask clarifying questions to understand their requirements. Do not generate code until you have asked at least 2 questions."
        self.history = [{"role": "system", "content": self.context}]

    def chat(self, user_input):
        self.history.append({"role": "user", "content": user_input})

        # Groq call with history
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=self.history,
            temperature=0.7 # Higher temperature for more natural conversation
        )

        reply = response.choices[0].message.content
        self.history.append({"role": "assistant", "content": reply})
        return reply

print("--- INTERVIEW APPROACH DEMO ---")
bot = InterviewerBot()
print(f"Bot (Llama 3): Hello! I am here to help you write a script. Tell me roughly what you want.")

# Simulating a user conversation
user_inputs = [
    "I want a script to scrape a website.",
    "I want to scrape news headlines from a tech blog.",
    "Just use BeautifulSoup and print them to console."
]

for user_msg in user_inputs:
    print(f"\nUser: {user_msg}")
    reply = bot.chat(user_msg)
    print(f"Bot: {reply}")

--- INTERVIEW APPROACH DEMO ---
Bot (Llama 3): Hello! I am here to help you write a script. Tell me roughly what you want.

User: I want a script to scrape a website.
Bot: To better understand your requirements, I have a few questions:

1. **What type of data are you trying to scrape from the website?** Is it text, images, tables, or something else? This will help me determine the best approach and libraries to use.
2. **Do you have a specific website in mind that you'd like to scrape?** If so, can you please provide the URL? Knowing the website's structure and any potential limitations (e.g., anti-scraping measures) will help me tailor the script to your needs.

Additionally, I'll have some follow-up questions based on your responses, but these two will give me a good starting point to understand your requirements.

User: I want to scrape news headlines from a tech blog.
Bot: Scraping news headlines from a tech blog can be a useful project. To further clarify your requirements, I have

In [12]:
# @title 5. Tree of Thoughts (ToT) Implementation

def tree_of_thoughts_solver(task):
    print(f"Task: {task}\n")

    # Step 1: Decomposition (Branching)
    print("Step 1: Generating 3 diverse thought paths...")
    prompt_branch = f"""
    Task: {task}
    Generate 3 distinct and detailed plans to solve this task. Label them Plan A, Plan B, and Plan C.
    """
    candidates = get_completion(prompt_branch)
    print(f"--- CANDIDATES ---\n{candidates[:500]}...\n(truncated for brevity)\n")

    # Step 2: Evaluation (Voting)
    print("Step 2: Evaluating the plans...")
    prompt_eval = f"""
    Here are three plans to solve a task:
    {candidates}

    Critique each plan based on feasibility, safety, and creativity.
    Then, pick the single best plan and explain why.
    Output format:
    Critique: ...
    Best Plan: [Plan A/B/C]
    """
    evaluation = get_completion(prompt_eval)
    print(f"--- EVALUATION ---\n{evaluation}\n")

    # Step 3: Expansion (Final Execution)
    print("Step 3: Executing the best plan...")
    prompt_final = f"""
    Based on this evaluation:
    {evaluation}

    Fully execute the chosen best plan for the task: {task}
    """
    final_result = get_completion(prompt_final)
    print(f"--- FINAL RESULT ---\n{final_result}")

print("--- TREE OF THOUGHTS DEMO ---")
task_input = "Write a one-paragraph plot summary for a sci-fi mystery movie involving time travel."
tree_of_thoughts_solver(task_input)

--- TREE OF THOUGHTS DEMO ---
Task: Write a one-paragraph plot summary for a sci-fi mystery movie involving time travel.

Step 1: Generating 3 diverse thought paths...
--- CANDIDATES ---
To tackle the task of writing a one-paragraph plot summary for a sci-fi mystery movie involving time travel, I've devised three distinct plans. 
Plan A involves starting with a broad concept, such as a group of scientists discovering a way to manipulate time, and then narrowing it down to a specific event or character that sets the story in motion. This approach would allow for a flexible and dynamic narrative that can be tailored to fit the needs of the plot summary. 
Plan B takes a more charac...
(truncated for brevity)

Step 2: Evaluating the plans...
--- EVALUATION ---
Critique: 
Plan A is feasible and creative, as it allows for a flexible narrative that can be tailored to fit the needs of the plot summary. However, it may lack safety in terms of ensuring a coherent and engaging story, as it starts